In [1]:
!pip install cova
!pip install pennylane
!pip install pennylane-sf

     |████████████████████████████████| 914 kB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 29.7 MB/s eta 0:00:01
     |████████████████████████████████| 8.7 MB 37.6 MB/s eta 0:00:01
     |████████████████████████████████| 41.6 MB 36.1 MB/s eta 0:00:01     |██████████                      | 12.9 MB 25.2 MB/s eta 0:00:02     |███████████████████████████████▉| 41.5 MB 36.1 MB/s eta 0:00:01
     |████████████████████████████████| 48 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 53.6 MB/s eta 0:00:01


  Using cached future-0.18.2.tar.gz (829 kB)
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=d8efd209903a57407792062fdc73987ee78078b9d34f84a15413dabdfaf0ac2f
  Stored in directory: /home/ram/.cache/pip/wheels/8e/70/28/3d6ccd6e315f65f245da085482a2e1c7d14b90b30f239e2cf4
Successfully built future
     |████████████████████████████████| 4.9 MB 6.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 15.6 MB/s eta 0:00:01     |██████████████████████████████  | 6.0 MB 15.6 MB/s eta 0:00:01
     |████████████████████████████████| 47 kB 14.9 MB/s eta 0:00:01
     |████████████████████████████████| 3.4 MB 35.7 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 7.0 MB/s  eta 0:00:01
     |████████████████████████████████| 112 kB 17.8 MB/s eta 0:00:01
     |████████████████████████████████| 103 kB 31.4 MB/s eta 0:00:01
     |████████████████████████████████| 532 kB 44.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 38

  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=5eb4f219320ff9d36086bd0592c8c76ac54d8e00619c8387db976d08b25c501a
  Stored in directory: /home/ram/.cache/pip/wheels/c8/d0/ab/d43c02eaddc5b9004db86950802442ad9a26f279c619e28da0
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=ffc9815b3bfdd6cf6c0b2e483d61eff90dce8911236f20d03b0131332defba2f
  Stored in directory: /home/ram/.cache/pip/wheels/1f/10/06/2a990ee4d73a8479fe2922445e8a876d38cfbfed052284c6a1
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=c02df7ee4060ba293439766046fd6f14c6f722ddd7c942810d10a2b2a03629b3
  Stored in directory: /home/ram/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
Successfully built antlr4-python3-runtime fire termcolor


In [54]:
!covalent start
!covalent purge

Covalent server has started at http://0.0.0.0:48008
Covalent server was not running.
Covalent server files have been purged.


In [288]:
import pennylane as qml
from pennylane import numpy as np
import covalent as ct
import matplotlib.pyplot as plt
from pennylane.optimize import NesterovMomentumOptimizer

In [289]:
dev = qml.device("default.qubit", wires=4)

In [290]:
@ct.electron
def layer(W):

    qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
    qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
    qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
    qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[3, 0])

In [291]:
@ct.electron
def statepreparation(x):
    qml.BasisState(x, wires=[0, 1, 2, 3])

In [292]:
@ct.electron
@qml.qnode(dev)
def circuit(weights, x):

    qml.BasisState(x, wires=[0, 1, 2, 3])

    for W in weights:
        qml.Rot(W[0, 0], W[0, 1], W[0, 2], wires=0)
        qml.Rot(W[1, 0], W[1, 1], W[1, 2], wires=1)
        qml.Rot(W[2, 0], W[2, 1], W[2, 2], wires=2)
        qml.Rot(W[3, 0], W[3, 1], W[3, 2], wires=3)

        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[1, 2])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[3, 0])

    return qml.expval(qml.PauliZ(0))

In [293]:
@ct.electron
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [294]:
@ct.electron
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [295]:
@ct.electron
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [296]:
@ct.electron
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [297]:
@ct.electron
def get_optimizer():
    return NesterovMomentumOptimizer(0.25)

In [298]:
@ct.electron
def weights_bias_init(num_layers,num_qubits):
    weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
    bias_init = np.array(0.0, requires_grad=True)
    return weights_init,bias_init

In [299]:
@ct.electron
def training(opt,weights,bias,epochs,batch_size,X,Y,num_layers,num_qubits,cost):
    #weights,bias = weights_bias_init(num_layers,num_qubits)
    training_steps = []
    cost_steps = []
    accuracy_steps = []
    for it in range(epochs):
        batch_index = np.random.randint(0, len(X), (batch_size,))
        X_batch = X[batch_index]
        Y_batch = Y[batch_index]
        weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

        # Compute accuracy
        predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
        acc = accuracy(Y, predictions)
        training_steps.append(i)
        cost_steps.append(cost(weights, bias, X, Y))
        accuracy_steps.append(acc)
        
        print("Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(it + 1, cost(weights, bias, X, Y), acc))
    
    return weights,bias,training_steps,cost_steps,accuracy_steps

In [300]:
@ct.lattice
def workflow(epochs,num_layers,num_qubits,X,Y):
    opt = get_optimizer()
    weights,bias = weights_bias_init(num_layers,num_qubits)
    batch_size = 5
    weights,bias,training_steps,cost_steps,accuracy_steps = training(opt,weights,bias,epochs,batch_size,X,Y,num_layers,num_qubits,cost)
    return weights,bias,training_steps,cost_steps,accuracy_steps
    

In [301]:
data = np.loadtxt("./parity.txt")
X = np.array(data[:, :-1], requires_grad=False)
Y = np.array(data[:, -1], requires_grad=False)
Y = Y * 2 - np.ones(len(Y))  # shift label from {0, 1} to {-1, 1}

for i in range(5):
    print("X = {}, Y = {: d}".format(X[i], int(Y[i])))

print("...")

X = [0. 0. 0. 0.], Y = -1
X = [0. 0. 0. 1.], Y =  1
X = [0. 0. 1. 0.], Y =  1
X = [0. 0. 1. 1.], Y = -1
X = [0. 1. 0. 0.], Y =  1
...


In [302]:

dispatch_id = ct.dispatch(workflow)(X=X,Y=Y,epochs=25,num_layers=2,num_qubits=4)
result = ct.get_result(dispatch_id=dispatch_id,wait=True,results_dir="./")
weights,bias,training_steps,cost_steps,accuracy_steps = result.result


KeyboardInterrupt: 

In [ ]:
plt.plot(training_steps, accuracy_steps)
plt.xlabel("Step")
plt.ylabel("Accuracy")
plt.show()